In [70]:
# Set simulation parameters
import ipywidgets as widgets
from IPython.core.display import HTML

style = {'description_width': '50px', 'min_width': '1000px'}

bar_widget = widgets.Dropdown(
    description='Bar:', value='TRAVERSAL', options=['TRAVERSAL', 'HIGH', 'MID', 'LOW'], style=style)
robot_mass_widget = widgets.FloatSlider(
    description='Robot Mass (lbs):', value=100, min=25, max=125, style=style)
robot_suspended_height_widget = widgets.FloatSlider(
    description='Robot Suspended Height (in.):', value=60.0, min=0, max=66.0, style=style)

trident_mass_widget = widgets.FloatSlider(
    description='Trident Mass (lbs):', value=10, min=0, max=50, style=style)
trident_height_widget = widgets.FloatSlider(
    description='Trident Height (in.):', value=66.0, min=0, max=66.0, style=style)
engage_overlap_widget = widgets.FloatSlider(
    description='Engage Overlap (in.):', value=6.0, min=1.75, max=12.0, style=style)

trident_robot_spring_constant_widget = widgets.FloatSlider(
    description='Spring Constant (lbsf/in.):', value=10, min=0, max=1000, style=style)


BAR_HEIGHTS = {
    'TRAVERSAL': 7.0*12 + 7.0,
    'HIGH': 6.0*12 + 3.625,
    'MID': 5.0*12 + 0.25,
    'LOW': 4.0*12 + 0.75,
}

G = 32.1740*12

interface_box = widgets.VBox((
    bar_widget,
    robot_mass_widget,
    robot_suspended_height_widget,
    trident_mass_widget,
    trident_height_widget,
    engage_overlap_widget,
    trident_robot_spring_constant_widget,
))


display(interface_box)

In [71]:
def simulation_solver(bar, robot_mass, robot_suspended_height, trident_mass, trident_height, engage_overlap, trident_robot_spring_constant, dt=0.001, simulation_record=[]):
    t = 0
    robot_y = robot_suspended_height
    trident_y = trident_height
    min_robot_y = BAR_HEIGHTS[bar] + engage_overlap - trident_height

    robot_velocity = 0
    robot_acceleration = -G - (trident_robot_spring_constant * (robot_y - (trident_y - trident_height))  / robot_mass)
    trident_velocity = 0
    trident_acceleration =  trident_robot_spring_constant * (robot_y - (trident_y - trident_height)) / trident_mass - G

    trident_target_height = BAR_HEIGHTS[bar] + engage_overlap

    while trident_y - trident_height < robot_y and robot_y > min_robot_y:
        simulation_record.append({
            't': t,
            'trident_height': trident_y,
            'trident_velocity': trident_velocity,
            'trident_acceleration': trident_acceleration,
            'robot_height': robot_y,
            'robot_velocity': robot_velocity,
            'robot_acceleration': robot_acceleration,
        })
        
        new_trident_acceleration = trident_robot_spring_constant * (robot_y - (trident_y - trident_height)) / trident_mass - G
        d_trident_velocity = dt * (trident_acceleration + new_trident_acceleration) / 2
        d_trident_y = dt * (trident_velocity + d_trident_velocity / 2)

        new_robot_acceleration = -G - (trident_robot_spring_constant * (robot_y - (trident_y - trident_height))  / robot_mass)
        d_robot_velocity = dt * (robot_acceleration + new_robot_acceleration) / 2
        d_robot_y = dt * (robot_velocity + d_robot_velocity / 2)

        t += dt
        trident_acceleration = new_trident_acceleration
        trident_velocity += d_trident_velocity
        trident_y += d_trident_y
        robot_acceleration = new_robot_acceleration
        robot_velocity += d_robot_velocity
        robot_y += d_robot_y
    
    print(f'Fall Distance                   : {robot_suspended_height - robot_y:.4f} in.')
    print(f'Fall Time                       : {t:.4f}s')

    print(f'Trident Max Height              : {trident_y:.4f} in.')
    print(f'Trident Target Height (in.)     : {trident_target_height:.4f} in.')

    passed = trident_y > trident_target_height and trident_height < BAR_HEIGHTS[bar]
    print(f'Configuration will {"" if passed else "fail to "}reach the {bar} rung.')
    return True


In [72]:
for bar_name in BAR_HEIGHTS:
    print('--------------------------------------------------------')
    simulation_solver(bar_name, robot_mass=robot_mass_widget.value, robot_suspended_height=robot_suspended_height_widget.value, trident_mass=trident_mass_widget.value, trident_height=trident_height_widget.value, engage_overlap=engage_overlap_widget.value, trident_robot_spring_constant=trident_robot_spring_constant_widget.value)
print('--------------------------------------------------------')

--------------------------------------------------------
Fall Distance                   : 29.0527 in.
Fall Time                       : 0.3850s
Trident Max Height              : 41.7733 in.
Trident Target Height (in.)     : 97.0000 in.
Configuration will fail to reach the TRAVERSAL rung.
--------------------------------------------------------
Fall Distance                   : 44.4050 in.
Fall Time                       : 0.4760s
Trident Max Height              : 28.9193 in.
Trident Target Height (in.)     : 81.6250 in.
Configuration will fail to reach the HIGH rung.
--------------------------------------------------------
Fall Distance                   : 59.9268 in.
Fall Time                       : 0.5530s
Trident Max Height              : 15.8872 in.
Trident Target Height (in.)     : 66.2500 in.
Configuration will fail to reach the MID rung.
--------------------------------------------------------
Fall Distance                   : 71.4841 in.
Fall Time                       : 0.60

In [73]:
from matplotlib import pyplot as plt
%matplotlib inline

def plot_simulation(bar, **kwargs):
    simulation_record = []
    simulation_solver(bar, simulation_record=simulation_record, **kwargs)

    times = [frame['t'] for frame in simulation_record]
    trident_heights = [frame['trident_height'] for frame in simulation_record]
    trident_velocities = [frame['trident_velocity'] for frame in simulation_record]
    trident_accelerations = [frame['trident_acceleration'] for frame in simulation_record]
    robot_heights = [frame['robot_height'] for frame in simulation_record]


    plt.plot([times[i//2] for i in range(2*len(times))], [trident_heights[i//2] - (i%2) * kwargs['trident_height'] for i in range(2*len(trident_heights))], color='cornflowerblue')
    plt.plot(times, trident_velocities, color='blue')
    plt.plot(times, trident_accelerations, color='cyan')
    plt.plot(times, robot_heights, color='darkorange')
    plt.plot(times, [BAR_HEIGHTS[bar]] * len(times), color='darkgray')
    plt.plot(times, [BAR_HEIGHTS[bar] + kwargs['engage_overlap'] - kwargs['trident_height']] * len(times), color='red')
    plt.ylim([0, 108])
    plt.show()

In [74]:
widgets.interactive(plot_simulation, dt=0.001, bar=bar_widget, robot_mass=robot_mass_widget, robot_suspended_height=robot_suspended_height_widget, trident_mass=trident_mass_widget, trident_height=trident_height_widget, engage_overlap=engage_overlap_widget, trident_robot_spring_constant=trident_robot_spring_constant_widget)

interactive(children=(Dropdown(description='Bar:', options=('TRAVERSAL', 'HIGH', 'MID', 'LOW'), style=Descript…

In [37]:
# # Calculate the interaction kinematically
# def kinematic_solver(bar: str, robot_mass, robot_suspended_height, trident_mass, trident_height, engage_overlap, trident_robot_force):

#     min_robot_height = BAR_HEIGHTS[bar] + engage_overlap - trident_height
#     robot_fall_displacement =  min_robot_height - robot_suspended_height
#     # h = a/2 * t^2
#     # t = sqrt(2h/a)
#     robot_net_acceleration = -G - trident_robot_force / robot_mass
#     t = (2 * robot_fall_displacement / robot_net_acceleration)**0.5
#     print(f'Fall Distance               : {robot_fall_displacement:.4f} in.')
#     print(f'Fall Time                   : {t:.4f}s')

#     # h = a/2 * t^2 + h_0
#     trident_net_acceleration = trident_robot_force / trident_mass - G
#     trident_end_height = trident_net_acceleration / 2 * t*t + trident_height
#     trident_target_height = BAR_HEIGHTS[bar] + engage_overlap
#     print(f'Trident Max Height          : {trident_end_height:.4f} in.')
#     print(f'Trident Target Height       : {trident_target_height:.4f} in.')

#     passed = trident_end_height > trident_target_height and trident_height < BAR_HEIGHTS[bar]
#     print(f'Configuration will {"" if passed else "fail to "}reach the {bar} rung.')

#     return passed

# widgets.interactive(kinematic_solver, bar=bar, robot_mass=robot_mass, robot_suspended_height=robot_suspended_height, trident_mass=trident_mass, trident_height=trident_height, engage_overlap=engage_overlap, trident_robot_spring_constant=trident_robot_spring_constant)